In [ ]:
import numpy as np
import cv2
import time
from utils import *  # Importing all the utility functions from utils.py

# Scoring System
lane_score = 100
score_deduction = 1
score_buffer = []
score_window = 50
alert_duration = 3
last_alert_time = 0
alert_counters = {"left": 0, "right": 0}
last_alert = None

frameWidth = 640
frameHeight = 480
noOfArrayValues = 10
arrayCounter = 0
arrayCurve = np.zeros([noOfArrayValues])

# Initialize Trackbars
initializeTrackbars([24, 55, 12, 100])

def process_frame(frame):
    global lane_score, last_alert, score_buffer, last_alert_time, alert_counters, arrayCounter, arrayCurve
    height, width, _ = frame.shape
    center_x = width // 2

    # Apply the thresholding from utils.py
    imgThres, imgCanny, imgColor = thresholding(frame)

    # Get source points for warping
    src = valTrackbars()

    # Apply perspective warp
    imgWarp = perspective_warp(imgThres, dst_size=(frameWidth, frameHeight), src=src)

    # Sliding window to detect lanes
    imgSliding, curves, lanes, ploty = sliding_window(imgWarp, draw_windows=True)

    # Draw lanes on the original frame
    if curves is not None:
        frame = draw_lanes(frame, curves[0], curves[1], frameWidth, frameHeight, src)
        lane_curve = np.mean([curves[0], curves[1]])
    else:
        lane_curve = 0

    # Lane score calculation
    lane_score = max(50, min(100, lane_score))
    score_buffer.append(lane_score)
    if len(score_buffer) > score_window:
        score_buffer.pop(0)

    average_score = int(np.mean(score_buffer))
    cv2.putText(frame, f"Lane Score: {average_score}", (width - 250, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2, cv2.LINE_AA)

    return frame

def update_alert(side, frame, width, height):
    global lane_score, last_alert_time, alert_counters, last_alert
    current_time = time.time()

    if last_alert_time == 0 or (current_time - last_alert_time > alert_duration):
        alert_counters[side] += 1
        lane_score = max(0, lane_score - score_deduction)
        last_alert_time = current_time
        last_alert = side

    multiplier_text = f" ({alert_counters[side]}x)" if alert_counters[side] > 1 else ""

    if side == "left":
        cv2.putText(frame, f"Too close to the left lane!{multiplier_text}", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
    elif side == "right":
        cv2.putText(frame, f"Too close to the right lane!{multiplier_text}", (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)

# Load your video
video_path = '/content/testdrive.mp4'
cap = cv2.VideoCapture(video_path)

fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Save the video properly
output_path = '/content/output_video.avi'
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

# Process video frames and write to file
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    processed_frame = process_frame(frame)
    out.write(processed_frame)

cap.release()
out.release()
print("Processing completed and video saved successfully.")